In [1]:
import utility_main
import pandas as pd
import urllib.request
import json 
from bs4 import BeautifulSoup
import requests
import json
from urllib.request import Request, urlopen
from urllib.error import URLError, HTTPError
from datetime import datetime, timedelta, date
import pytz
import dateutil.parser
from collections import defaultdict
import random
import json
import csv
from dateutil.parser import parse
import os
import datetime

The file: /non existentnon_existent_file.json does not exist
From convert_json_to_df(): Could not convert the json file to dataframe
From preprocess_df(): Check the dataframe object and column names


In [55]:
def cbc_sampling_wrapper(economic_indicator,project_path, sample_size,column_names_list = ['title', 'description','publishedAt'],source_string = "cbc", remove_Nans = True):
    """This function creates a sample from a given file  of json 
    Input:
    ------
    economic_indicator: The economic indicator
    project_path: The path of the project
    json_file_path: The path tto the file which has the article jsons
    column_names_list: The list of column names to retain in the final sampled file
    sample_size: The number of articles to sample per month in any year
    source_string: The source (CBC/Bloomberg)"""
    try:
        economic_indicator_list = economic_indicator.split(" ")
        economic_indicator="".join(economic_indicator_list).lower()
        json_file_path =  "data_extraction/data/unannotated_data/" + source_string.upper() + "/" +  economic_indicator + "_" + source_string.upper() + "_article.json"
        print("Reading json file :", project_path + json_file_path + "\n")
        
        df = utility_main.convert_json_to_df(project_path, json_file_path)
        
        preprocessed_df = utility_main.preprocess_df(df,column_names_list)
        preprocessed_df["source"] = source_string
        preprocessed_df = utility_main.apply_lambda(preprocessed_df,'publishedAt' ,"lambda x: x[1]")
        
        sampled_df = utility_main.sample_dataframe_by_month(preprocessed_df, sample_size)
        sampled_df = utility_main.apply_lambda(sampled_df,'publishedAt',"lambda x: dateutil.parser.isoparse(x).date()")
        
        # Write sampled dataframe to data_extraction/data/annotated_data/<source>/<econimc_indicator_sample.csv>
        output_file_path = "data_extraction/data/annotated_data/"  + source_string + "/"
        utility_main.write_df_to_csv(sampled_df,project_path,output_file_path,economic_indicator+"_sample.csv")
        
        
        if utility_main.file_exists(project_path + output_file_path +economic_indicator+"_sample.csv"):
            print("Sample file generated at: ", project_path + output_file_path +economic_indicator+"_sample.csv \n")
            x = utility_main.get_unannotated_data(preprocessed_df, sampled_df, economic_indicator, source_string, project_path)
            if x == True:
                print("Sample and predictions ready \n")
                return True
            else:
                print("Failed to generate predictions file")
                return False
        else:
            print("Something went wrong")
    except:
        print("Could not successfully sample the data")
        return False
    


In [56]:
def unit_tests():
    # Testing with wrong project path
    project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/"
    rt = cbc_sampling_wrapper("mortgagerates",project_path, 2,"cbc")
    assert rt == False, "function should return False if wrong file path/name is given"
    
    # Testing with wrong column names
    project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/"
    rt = cbc_sampling_wrapper("mortgagerates",project_path,2,['xts', 'description','publishedAt'],"cbc")
    assert rt == False, "function should return False if wrong column names is given"
    # Give wrong sample size
    project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/"
    rt = cbc_sampling_wrapper("mortgagerates",project_path, 0, ['title', 'description','publishedAt'],"cbc")
    assert rt == False, "function should return false if sample size is 0 or less"
    print("Successfully passed tests")

unit_tests()

Reading json file : /Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/data_extraction/data/unannotated_data/CBC/mortgagerates_CBC_article.json

The file: /Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/data_extraction/data/unannotated_data/CBC/mortgagerates_CBC_article.json does not exist
From convert_json_to_df(): Could not convert the json file to dataframe
From preprocess_df(): Check the dataframe object and column names
Could not successfully sample the data
Reading json file : /Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/data_extraction/data/unannotated_data/CBC/mortgagerates_CBC_article.json

From preprocess_df(): Check the dataframe object and column names
Could not successfully sample the data
Reading json file : /Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/data_extraction/data/unannotated_data/CBC/mortgagerates_CBC_article.json

Sample size should be greater than 0
Cannot apply lambda function
Could not 

In [57]:
# project_path = "/Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/"
project_path = "../../../"
cbc_sampling_wrapper("Housing",project_path,3)

Reading json file : ../../../data_extraction/data/unannotated_data/CBC/housing_CBC_article.json

Sample file generated at:  ../../../data_extraction/data/annotated_data/cbc/housing_sample.csv 

Predictions file generated at : ../../../sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_housing_cbc.csv
Sample and predictions ready 



True

In [50]:
cbc_sampling_wrapper("mortgage rates",project_path, 3)

Reading json file : ../../../data_extraction/data/unannotated_data/CBC/mortgagerates_CBC_article.json

Sample file generated at:  ../../../data_extraction/data/annotated_data/cbc/mortgagerates_sample.csv 

Predictions file generated at : ../../../sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_mortgagerates_cbc.csv
Sample and predictions ready 



True

In [52]:
cbc_sampling_wrapper("interest rates",project_path, 3)

Reading json file : ../../../data_extraction/data/unannotated_data/CBC/interestrates_CBC_article.json

Sample file generated at:  ../../../data_extraction/data/annotated_data/cbc/interestrates_sample.csv 

Predictions file generated at : ../../../sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_interestrates_cbc.csv
Sample and predictions ready 



True

In [58]:
cbc_sampling_wrapper("tsx",project_path, 3)

Reading json file : ../../../data_extraction/data/unannotated_data/CBC/tsx_CBC_article.json

Sample file generated at:  ../../../data_extraction/data/annotated_data/cbc/tsx_sample.csv 

Predictions file generated at : ../../../sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_tsx_cbc.csv
Sample and predictions ready 



True

In [59]:
cbc_sampling_wrapper("gdp",project_path, 3)

Reading json file : ../../../data_extraction/data/unannotated_data/CBC/gdp_CBC_article.json

Sample file generated at:  ../../../data_extraction/data/annotated_data/cbc/gdp_sample.csv 

Predictions file generated at : ../../../sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_gdp_cbc.csv
Sample and predictions ready 



True

In [60]:
cbc_sampling_wrapper("employment",project_path, 3)

Reading json file : ../../../data_extraction/data/unannotated_data/CBC/employment_CBC_article.json

Sample file generated at:  ../../../data_extraction/data/annotated_data/cbc/employment_sample.csv 

Predictions file generated at : ../../../sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_employment_cbc.csv
Sample and predictions ready 



True

In [3]:
# To sample only those articles with given key word use the code below and change parameters
# utility_main.get_keyword_df("mortgagerates",project_path, ["mortgage", "rent", "housing", "interest"], "predictions_dataset_mortgagerates_cbc.csv", "cbc")

Total articles retrieved:  112
Outputted filtered articles to predict to file : /Users/nagasiri/Desktop/NagaSiri/MDS-CL/Capstone/better_dwelling_capstone/project/sentiment_analyzer/data/predictions_data/cbc/predictions_dataset_mortgagerates_cbc.csv
